## Stochastic Block Model Experiment

Two nodes within the same cluster of the empirical graph are connected by an edge with probability pin, two nodes from different clusters are connected by an edge with probability pout. Each node $i \in V$ represents a local dataset consisting of 5 feature vectors $x^{(i,1)}, ... , x^{(i,5)} \in R^2$. The feature vectors are i.i.d. realizations of a standard Gaussian random vector x ∼ N(0,I). The labels $y_1^{(i)}, . . . , y_5^{(i)} \in R$ of the nodes $i \in V$ are generated according to the linear model $y_r^{(i)} = (x^{(i, r)})^T w^{(i)} + \epsilon$, with $\epsilon ∼ N(0,\sigma)$. To learn the weight $w^{(i)}$ ,we apply Algorithm 1 to a training set M obtained by randomly selecting 40% of the nodes.

Before get into the experiment more, lets review algorithm 1 and the primal and dual updates.

### Algorithm 1

![title](algorithm1.png)

In [2]:
# %load algorithm/main.py
from sklearn.metrics import mean_squared_error

from algorithm.penalty import *


def algorithm_1(K, D, weight_vec, datapoints, true_labels, samplingset, lambda_lasso, penalty_func_name='norm1', calculate_score=False):
    '''
    :param K: the number of iterations
    :param D: the block incidence matrix
    :param weight_vec: a list containing the edges's weights of the graph
    :param datapoints: a dictionary containing the data of each node in the graph needed for the algorithm 1
    :param true_labels: a list containing the true labels of the nodes
    :param samplingset: the sampling set
    :param lambda_lasso: the parameter lambda
    :param penalty_func_name: the name of the penalty function used in the algorithm

    :return iteration_scores: the mean squared error of the predicted weight vectors in each iteration
    :return new_w: the predicted weigh vectors for each node
    '''

    Sigma = np.diag(np.full(weight_vec.shape, 0.9 / 2))
    '''
    Sigma: the block diagonal matrix Sigma
    '''
    T_matrix = np.diag(np.array((1.0 / (np.sum(abs(D), 0)))).ravel())
    '''
    T_matrix: the block diagonal matrix T
    '''

    if np.linalg.norm(np.dot(Sigma ** 0.5, D).dot(T_matrix ** 0.5), 2) > 1:
        print ('product norm', np.linalg.norm(np.dot(Sigma ** 0.5, D).dot(T_matrix ** 0.5), 2))

    E, N = D.shape
    m, n = datapoints[0]['features'].shape

    # define the penalty function
    if penalty_func_name == 'norm1':
        penalty_func = Norm1Pelanty(lambda_lasso, weight_vec, Sigma, n)

    elif penalty_func_name == 'norm2':
        penalty_func = Norm2Pelanty(lambda_lasso, weight_vec, Sigma, n)

    elif penalty_func_name == 'mocha':
        penalty_func = MOCHAPelanty(lambda_lasso, weight_vec, Sigma, n)

    else:
        raise Exception('Invalid penalty name')

    # starting algorithm 1

    new_w = np.array([np.zeros(n) for i in range(N)])
    '''
    new_w: the primal variable of the algorithm 1
    '''
    new_u = np.array([np.zeros(n) for i in range(E)])
    '''
    new_u: the dual variable of the algorithm 1
    '''

    iteration_scores = []
    for iterk in range(K):
        # if iterk % 100 == 0:
        #     print ('iter:', iterk)
        prev_w = np.copy(new_w)

        # algorithm 1, line 2
        hat_w = new_w - np.dot(T_matrix, np.dot(D.T, new_u))

        for i in range(N):
            if i in samplingset:  # algorithm 1, line 6

                optimizer = datapoints[i]['optimizer']
                new_w[i] = optimizer.optimize(datapoints[i]['features'], datapoints[i]['label'], hat_w[i], datapoints[i]['degree'])

            else:
                new_w[i] = hat_w[i]

        # algorithm 1, line 9
        tilde_w = 2 * new_w - prev_w
        new_u = new_u + np.dot(Sigma, np.dot(D, tilde_w))

        # algorithm 1, line 10
        new_u = penalty_func.update(new_u)

        # calculate the MSE of the predicted weight vectors
        if calculate_score:
            Y_pred = []
            for i in range(N):
                Y_pred.append(np.dot(datapoints[i]['features'], new_w[i]))

            iteration_scores.append(mean_squared_error(true_labels.reshape(N, m), Y_pred))

    # print (np.max(abs(new_w - prev_w)))

    return iteration_scores, new_w


### Primal Update 

As you see in the algorithm picture, the primal update needs a optimizer operator for the sampling set (line 6). We have implemented the optimizers discussed in the paper, both the logistic loss and squared error loss optimizers implementations with pytorch is available, also we have implemented the squared error loss optimizer using the fixed point equation in the `Networked Linear Regression` section of the paper.  

In [3]:
# %load algorithm/optimizer.py 
import torch
import abc
import numpy as np

from abc import ABC


# The linear model which is implemented by pytorch
class TorchLinearModel(torch.nn.Module):
    def __init__(self, n):
        super(TorchLinearModel, self).__init__()
        self.linear = torch.nn.Linear(n, 1, bias=False)

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred


# The abstract optimizer model which should have model, optimizer, and criterion as the input
class Optimizer(ABC):
    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    @abc.abstractmethod
    def optimize(self, x_data, y_data, old_weight, regularizer_term):
        torch_old_weight = torch.from_numpy(np.array(old_weight, dtype=np.float32))
        self.model.linear.weight.data = torch_old_weight
        for iterinner in range(40):
            self.optimizer.zero_grad()
            y_pred = self.model(x_data)
            loss1 = self.criterion(y_pred, y_data)
            loss2 = 1 / (2 * regularizer_term) * torch.mean((self.model.linear.weight - torch_old_weight) ** 2)  # + 10000*torch.mean((model.linear.bias+0.5)**2)#model.linear.weight.norm(2)
            loss = loss1 + loss2
            loss.backward()
            self.optimizer.step()

        return self.model.linear.weight.data.numpy()


# The linear model in Networked Linear Regression section of the paper
class LinearModel:
    def __init__(self, degree, features, label):
        mtx1 = 2 * degree * np.dot(features.T, features).astype('float64')
        mtx1 += 1 * np.eye(mtx1.shape[0])
        mtx1_inv = np.linalg.inv(mtx1)

        mtx2 = 2 * degree * np.dot(features.T, label).T

        self.mtx1_inv = mtx1_inv
        self.mtx2 = mtx2

    def forward(self, x):
        mtx2 = x + self.mtx2
        mtx_inv = self.mtx1_inv

        return np.dot(mtx_inv, mtx2)


# The Linear optimizer in Networked Linear Regression section of the paper
class LinearOptimizer(Optimizer):

    def __init__(self, model):
        super(LinearOptimizer, self).__init__(model, None, None)

    def optimize(self, x_data, y_data, old_weight, regularizer_term):
        return self.model.forward(old_weight)


# The Linear optimizer model which is implemented by pytorch
class TorchLinearOptimizer(Optimizer):
    def __init__(self, model):
        criterion = torch.nn.MSELoss(reduction='mean')
        optimizer = torch.optim.RMSprop(model.parameters())
        super(TorchLinearOptimizer, self).__init__(model, optimizer, criterion)

    def optimize(self, x_data, y_data, old_weight, regularizer_term):
        return super(TorchLinearOptimizer, self).optimize(x_data, y_data, old_weight, regularizer_term)


# The Logistic optimizer model which is implemented by pytorch
class TorchLogisticOptimizer(Optimizer):
    def __init__(self, model):
        criterion = torch.nn.BCELoss(reduction='mean')
        optimizer = torch.optim.RMSprop(model.parameters())
        super(TorchLogisticOptimizer, self).__init__(model, optimizer, criterion)

    def optimize(self, x_data, y_data, old_weight, regularizer_term):
        return super(TorchLogisticOptimizer, self).optimize(x_data, y_data, old_weight, regularizer_term)


### Dual Update 

As mentioned in the paper, the dual update has a penalty function(line 10) which is either norm1, norm2, or mocha.

In [4]:
# %load algorithm/penalty.py
import abc
import numpy as np

from abc import ABC


# The abstract penalty function which has a function update
class Penalty(ABC):
    def __init__(self, lambda_lasso, weight_vec, Sigma, n):
        self.lambda_lasso = lambda_lasso
        self.weight_vec = weight_vec
        self.Sigma = Sigma

    @abc.abstractmethod
    def update(self, new_u):
        pass


# The norm2 penalty function
class Norm2Pelanty(Penalty):
    def __init__(self, lambda_lasso, weight_vec, Sigma, n):
        super(Norm2Pelanty, self).__init__(lambda_lasso, weight_vec, Sigma, n)
        self.limit = np.array(lambda_lasso * weight_vec)

    def update(self, new_u):
        normalized_u = np.where(np.linalg.norm(new_u, axis=1) >= self.limit)
        new_u[normalized_u] = (new_u[normalized_u].T * self.limit[normalized_u] / np.linalg.norm(new_u[normalized_u], axis=1)).T
        return new_u


# The MOCHA penalty function
class MOCHAPelanty(Penalty):
    def __init__(self, lambda_lasso, weight_vec, Sigma, n):
        super(MOCHAPelanty, self).__init__(lambda_lasso, weight_vec, Sigma, n)
        self.normalize_factor = 1 + np.dot(2 * self.Sigma, 1/(self.lambda_lasso * self.weight_vec))

    def update(self, new_u):
        for i in range(new_u.shape[1]):
            new_u[:, i] /= self.normalize_factor

        return new_u


# The norm1 penalty function
class Norm1Pelanty(Penalty):
    def __init__(self, lambda_lasso, weight_vec, Sigma, n):
        super(Norm1Pelanty, self).__init__(lambda_lasso, weight_vec, Sigma, n)
        self.limit = np.array([np.zeros(n) for i in range(len(weight_vec))])
        for i in range(n):
            self.limit[:, i] = lambda_lasso * weight_vec

    def update(self, new_u):
        normalized_u = np.where(abs(new_u) >= self.limit)
        new_u[normalized_u] = self.limit[normalized_u] * new_u[normalized_u] / abs(new_u[normalized_u])
        return new_u


## Create SBM Data

In [8]:
from algorithm.optimizer import *
from torch.autograd import Variable
from graspy.simulations import sbm


def get_sbm_data(cluster_sizes, G, W, m=5, n=2, noise_sd=0, is_torch_model=True):
    '''
    :param cluster_sizes: a list containing the size of each cluster
    :param G: generated SBM graph with defined clusters using graspy.simulations
    :param W: a list containing the weight vectors for each cluster
    :param m, n: shape of features vector for each node
    :param pin: the probability of edges inside each cluster
    :param pout: the probability of edges between the clusters
    :param noise_sd: the standard deviation of the noise for calculating the labels
    
    :return B: adjacency matrix of the graph
    :return weight_vec: a list containing the edges's weights of the graph
    :return true_labels: a list containing the true labels of the nodes
    :return datapoints: a dictionary containing the data of each node in the graph needed for the algorithm 1 
    '''

    N = len(G)
    E = int(len(np.argwhere(G > 0))/2)
    '''
    N: total number of nodes
    E: total number of edges
    '''
    
    
    # create B(adjacency matrix) and edges's weights vector(weight_vec) based on the graph G
    B = np.zeros((E, N))
    '''
    B: adjacency matrix of the graph with the shape of E*N
    '''
    weight_vec = np.zeros(E)
    '''
    weight_vec: a list containing the edges's weights of the graph with the shape of E
    '''
    
    cnt = 0
    for i, j in np.argwhere(G > 0):
        if i > j:
            continue
        B[cnt, i] = 1
        B[cnt, j] = -1

        weight_vec[cnt] = 1
        cnt += 1
    
    
    # create the data of each node needed for the algorithm 1 
    
    node_degrees = np.array((1.0 / (np.sum(abs(B), 0)))).ravel()
    '''
    node_degrees: a list containing the nodes degree for the alg1 (1/N_i)
    '''
    
    datapoints = {}
    '''
    datapoints: a dictionary containing the data of each node in the graph needed for the algorithm 1,
    which are features, label, degree, and also the optimizer model for each node
    '''
    true_labels = []
    '''
    true_labels: the true labels for the nodes of the graph
    '''
    cnt = 0
    for i, cluster_size in enumerate(cluster_sizes):
        for j in range(cluster_size):
            features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
            '''
            features: the feature vector of node i which are i.i.d. realizations of a standard Gaussian random vector x~N(0,I)
            '''
            label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
            '''
            label: the label of the node i that is generated according to the linear model y = x^T w + e
            '''
            
            true_labels.append(label)

            if is_torch_model:
                model = TorchLinearModel(n)
                optimizer = TorchLinearOptimizer(model)
                features = Variable(torch.from_numpy(features)).to(torch.float32)
                label = Variable(torch.from_numpy(label)).to(torch.float32) 

            else:

                model = LinearModel(node_degrees[i], features, label)
                optimizer = LinearOptimizer(model)            
            '''
            model : the linear model for the node i 
            optimizer : the optimizer model for the node i 
            ''' 
            
            datapoints[cnt] = {
                'features': features,
                'degree': node_degrees[i],
                'label': label,
                'optimizer': optimizer
            }
            cnt += 1
        

    return B, weight_vec, np.array(true_labels), datapoints




### SBM with Two Clusters

The size of the clusters are {100, 100} with the weight vector $w^{(i)} = (2, 2)^T$ for $i \in C_1$ and $w^{(i)} = (−2,2)^T$ for $i \in C_2$. We run Algorithm 1 for different choices of $pout$ with a fixed $pin = 1/2$.

In [9]:
PENALTY_FUNCS = ['norm1', 'norm2', 'mocha']

K = 2000
M = 0.4
PIN = 0.5

In [10]:
from graspy.simulations import sbm


def get_sbm_2blocks_data(m=5, n=2, pin=0.5, pout=0.01, noise_sd=0, is_torch_model=True):
    '''
    :param m, n: shape of features vector for each node
    :param pin: the probability of edges inside each cluster
    :param pout: the probability of edges between the clusters
    :param noise_sd: the standard deviation of the noise for calculating the labels
    
    :return B: adjacency matrix of the graph
    :return weight_vec: a list containing the edges's weights of the graph
    :return true_labels: a list containing the true labels of the nodes
    :return datapoints: a dictionary containing the data of each node in the graph needed for the algorithm 1 
    '''
    cluster_sizes = [100, 100]

    # generate graph G which is a SBM wich 2 clusters
    G = sbm(n=cluster_sizes, p=[[pin, pout],[pout, pin]])
    '''
    G: generated SBM graph with 2 clusters
    ''' 
    
    # define weight vectors for each cluster of the graph
    
    W1 = np.array([2, 2])
    '''
    W1: the weigh vector for the first cluster
    '''
    W2 = np.array([-2, 2])
    '''
    W2: the weigh vector for the second cluster
    '''
    
    W = [W1, W2]
    
    
    return get_sbm_data(cluster_sizes, G, W, m, n, noise_sd, is_torch_model)



In [11]:
# %load regression_lasso/scores.py
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


def get_algorithm_1_scores(X, Y, new_w, samplingset, not_samplingset):
    Y_pred = []
    for i in range(len(X)):
        Y_pred.append(np.dot(X[i], new_w[i]))
    Y_pred = np.array(Y_pred)

    alg_1_score = {'total': mean_squared_error(Y, Y_pred),
                   'train': mean_squared_error(Y[samplingset], Y_pred[samplingset]),
                   'test': mean_squared_error(Y[not_samplingset], Y_pred[not_samplingset])}

    return alg_1_score


def get_linear_regression_score(x, y, decision_tree_samplingset, decision_tree_not_samplingset):
    model = LinearRegression().fit(x[decision_tree_samplingset], y[decision_tree_samplingset])
    pred_y = model.predict(x)
    linear_regression_score = {'total': mean_squared_error(y, pred_y),
                               'train': mean_squared_error(y[decision_tree_samplingset],
                                                           pred_y[decision_tree_samplingset]),
                               'test': mean_squared_error(y[decision_tree_not_samplingset],
                                                          pred_y[decision_tree_not_samplingset])}

    return linear_regression_score


def get_decision_tree_score(x, y, decision_tree_samplingset, decision_tree_not_samplingset):
    max_depth = 2
    regressor = DecisionTreeRegressor(max_depth=max_depth)
    regressor.fit(x[decision_tree_samplingset], y[decision_tree_samplingset])
    pred_y = regressor.predict(x)
    decision_tree_score = {'total': mean_squared_error(y, pred_y),
                           'train': mean_squared_error(y[decision_tree_samplingset],
                                                       pred_y[decision_tree_samplingset]),
                           'test': mean_squared_error(y[decision_tree_not_samplingset],
                                                      pred_y[decision_tree_not_samplingset])}
    return decision_tree_score


def get_scores(datapoints, new_w, samplingset):
    X = []
    Y = []
    for i in range(len(datapoints)):
        X.append(np.array(datapoints[i]['features']))
        Y.append(np.array(datapoints[i]['label']))

    X = np.array(X)
    Y = np.array(Y)

    N = len(X)
    m, n = X[0].shape

    not_samplingset = [i for i in range(N) if i not in samplingset]

    alg_1_score = get_algorithm_1_scores(X, Y, new_w, samplingset, not_samplingset)

    y = Y.reshape(-1, 1)
    x = X.reshape(-1, n)
    decision_tree_samplingset = []
    for item in samplingset:
        for i in range(m):
            decision_tree_samplingset.append(m * item + i)
    decision_tree_not_samplingset = [i for i in range(m * N) if i not in decision_tree_samplingset]

    linear_regression_score = get_linear_regression_score(x, y, decision_tree_samplingset, decision_tree_not_samplingset)

    decision_tree_score = get_decision_tree_score(x, y, decision_tree_samplingset, decision_tree_not_samplingset)

    return alg_1_score, linear_regression_score, decision_tree_score


In [11]:
import random 


lambda_lasso = 0.01
POUT = 0.1

B, weight_vec, true_labels, datapoints = get_sbm_2blocks_data(pin=PIN, pout=POUT, is_torch_model=False)

samplingset = random.sample([i for i in range(N)], k=int(M* N))

for penalty_func in PENALTY_FUNCS:

    _, predicted_w = algorithm_1(K, B, weight_vec, datapoints, true_labels, samplingset, lambda_lasso, penalty_func)

    alg1_score, linear_regression_score, decision_tree_score = get_scores(datapoints, predicted_w, samplingset)
    print('penalty function:', penalty_func)
    print('algorithm 1 MSE:', alg1_score)
    print('algorithm 1 MSE:', alg1_score)
    print('linear regression MSE:', linear_regression_score)
    print('decision tree MSE:', decision_tree_score)
    print('------------------------')
        

penalty function: norm1
algorithm 1 MSE: {'total': 0.0007506122720691646, 'train': 0.0007697176093419437, 'test': 0.0007378753805539782}
algorithm 1 MSE: {'total': 0.0007506122720691646, 'train': 0.0007697176093419437, 'test': 0.0007378753805539782}
linear regression MSE: {'total': 3.675278029683866, 'train': 3.7491906805761324, 'test': 3.6260029290890223}
decision tree MSE: {'total': 3.8642290863200213, 'train': 3.819295840477632, 'test': 3.894184583548282}
------------------------
penalty function: norm2
algorithm 1 MSE: {'total': 0.0007646549100200437, 'train': 0.0007755543659338799, 'test': 0.0007573886060774864}
algorithm 1 MSE: {'total': 0.0007646549100200437, 'train': 0.0007755543659338799, 'test': 0.0007573886060774864}
linear regression MSE: {'total': 3.675278029683866, 'train': 3.7491906805761324, 'test': 3.6260029290890223}
decision tree MSE: {'total': 3.8642290863200213, 'train': 3.819295840477632, 'test': 3.894184583548281}
------------------------
penalty function: mocha


### SBM with Five Clusters

In [12]:
from graspy.simulations import sbm


def get_sbm_5blocks_data(m=5, n=2, pin=0.5, pout=0.01, noise_sd=0, is_torch_model=True):
    '''
    :param m, n: shape of features vector for each node
    :param pin: the probability of edges inside each cluster
    :param pout: the probability of edges between the clusters
    :param noise_sd: the standard deviation of the noise for calculating the labels
    
    :return B: adjacency matrix of the graph
    :return weight_vec: a list containing the edges's weights of the graph
    :return true_labels: a list containing the true labels of the nodes
    :return datapoints: a dictionary containing the data of each node in the graph needed for the algorithm 1 
    '''
    cluster_sizes = [70, 10, 50, 100, 150]
    
    p = [[pin if i==j else pout for i in range(len(cluster_sizes))] for j in range(len(cluster_sizes))]

    # generate graph G which is a SBM wich 2 clusters
    G = sbm(n=cluster_sizes, p=p)
    '''
    G: generated SBM graph with 2 clusters
    ''' 
    
    # define weight vectors for each cluster of the graph
    W = []
    for i in range(len(cluster_sizes)):
        # the weigh vector for the ith cluster
        W.append(np.random.random(n))
         
    
    
    return get_sbm_data(cluster_sizes, G, W, m, n, noise_sd, is_torch_model)




In [31]:
import random 


lambda_lasso = 0.01
POUT = 0.001

B, weight_vec, true_labels, datapoints = get_sbm_5blocks_data(pin=PIN, pout=POUT, is_torch_model=False)
E, N = B.shape

samplingset = random.sample([i for i in range(N)], k=int(0.2* N))

for penalty_func in PENALTY_FUNCS:

    _, predicted_w = algorithm_1(K, B, weight_vec, datapoints, true_labels, samplingset, lambda_lasso, penalty_func)

    alg1_score, linear_regression_score, decision_tree_score = get_scores(datapoints, predicted_w, samplingset)
    print('penalty function:', penalty_func)
    print('algorithm 1 MSE:', alg1_score)
    print('linear regression MSE:', linear_regression_score)
    print('decision tree MSE:', decision_tree_score)
    print('------------------------')
    

penalty function: norm1
algorithm 1 MSE: {'total': 9.803197971292387e-06, 'train': 9.794808579976995e-06, 'test': 9.805295319121232e-06}
linear regression MSE: {'total': 0.1090574069081943, 'train': 0.0908367184646107, 'test': 0.11361257901909018}
decision tree MSE: {'total': 0.2821424030956451, 'train': 0.20808492357075345, 'test': 0.30065677297686805}
------------------------
penalty function: norm2
algorithm 1 MSE: {'total': 4.8889451680751135e-06, 'train': 4.888762724191483e-06, 'test': 4.888990779046022e-06}
linear regression MSE: {'total': 0.1090574069081943, 'train': 0.0908367184646107, 'test': 0.11361257901909018}
decision tree MSE: {'total': 0.2821424030956451, 'train': 0.20808492357075345, 'test': 0.30065677297686805}
------------------------
penalty function: mocha
algorithm 1 MSE: {'total': 0.04372698239731128, 'train': 1.1564561098599744e-05, 'test': 0.05465583685636444}
linear regression MSE: {'total': 0.1090574069081943, 'train': 0.0908367184646107, 'test': 0.11361257901